In [1]:
from hypernodes import Pipeline, node


@node(output_name="doubled")
def double(x: int) -> int:
    return x * 2


@node(output_name="result")
def add_one(doubled: int) -> int:
    return doubled + 1


pipeline = Pipeline(nodes=[double, add_one], name="double_and_add")

In [2]:
result = pipeline.run(inputs={"x": 5})
result

{'doubled': 10, 'result': 11}

In [3]:
result = pipeline.map(inputs={"x": [1, 2, 3]}, map_over="x")
result

{'doubled': [2, 4, 6], 'result': [3, 5, 7]}

In [4]:
import time

from hypernodes import Pipeline, node
from hypernodes.telemetry import ProgressCallback


@node(output_name="doubled")
def double(x: int) -> int:
    time.sleep(0.1)  # Small delay to see progress
    return x * 2


@node(output_name="result")
def add_one(doubled: int) -> int:
    time.sleep(0.1)
    return doubled + 1


# Create pipeline with progress callback
progress = ProgressCallback(enable=True)  # Disable for tests
pipeline = Pipeline(
    nodes=[double, add_one], callbacks=[progress], name="double_and_add"
)

result = pipeline.run(inputs={"x": 5})

double_and_add   0%|          | 0/2 [00:00<?, ?it/s]

double   0%|          | 0/1 [00:00<?, ?it/s]

add_one   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
import logfire

from hypernodes import Pipeline, node
from hypernodes.telemetry import TelemetryCallback

# Configure logfire (local only, no cloud export)
logfire.configure(send_to_logfire=False)
telemetry = TelemetryCallback()

pipeline = Pipeline(
    nodes=[double, add_one], callbacks=[telemetry], name="double_and_add"
)

result = pipeline.run(inputs={"x": 5})

16:41:31.783 pipeline:pipeline_4401786720
16:41:31.784   node:double
16:41:31.890   node:add_one


In [6]:
chart = telemetry.get_waterfall_chart()
chart

In [7]:
from hypernodes import Pipeline, node
from hypernodes.telemetry import ProgressCallback


@node(output_name="encoded")
def encode_text(text: str) -> list:
    """Simulate text encoding."""
    time.sleep(0.3)
    return [1, 2, 3]


@node(output_name="embedded")
def embed_vectors(encoded: list) -> list:
    """Simulate vector embedding."""
    time.sleep(0.3)
    return [0.1, 0.2, 0.3]


@node(output_name="normalized")
def normalize(embedded: list) -> list:
    """Simulate normalization."""
    time.sleep(0.3)
    total = sum(embedded)
    return [x / total for x in embedded]


@node(output_name="scores")
def calculate_scores(normalized: list) -> float:
    """Simulate scoring."""
    time.sleep(0.3)
    return sum(normalized)


# Create pipeline with a meaningful name
progress = ProgressCallback(enable=True)
retrieval_pipeline = Pipeline(
    nodes=[encode_text, embed_vectors, normalize, calculate_scores],
    callbacks=[progress],
    name="retrieval pipeline",
)

# The progress bar will now show:
# "Running retrieval pipeline with N examples..." initially
# "retrieval pipeline → encode_text" when encoding
# "retrieval pipeline → embed_vectors" when embedding
# "retrieval pipeline → normalize" when normalizing
# "retrieval pipeline → calculate_scores" when scoring

results = retrieval_pipeline.map(
    inputs={"text": ["doc1", "doc2", "doc3", "doc4", "doc5"]}, map_over="text"
)

print(f"\nProcessed {len(results['scores'])} documents")

Running retrieval pipeline with 5 examples...   0%|          | 0/5 [00:00<?, ?it/s]

encode_text   0%|          | 0/5 [00:00<?, ?it/s]

embed_vectors   0%|          | 0/5 [00:00<?, ?it/s]

normalize   0%|          | 0/5 [00:00<?, ?it/s]

calculate_scores   0%|          | 0/5 [00:00<?, ?it/s]


Processed 5 documents
